In [55]:
#sys libs
import os
import sys
import random
import time
import re
import json

#data manupulation libs
import numpy as np

#PyTorch
import torch
from torch import nn
import torch.nn.functional as F
print('torch.__version__', torch.__version__)
NUM_GPUS = torch.cuda.device_count()
print('NUM_GPUS', NUM_GPUS)

# huggingface transformers
import transformers
print('transformers.__version__', transformers.__version__)
from transformers import AutoTokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import GPTJForCausalLM
from transformers import top_k_top_p_filtering

#fara
from fara.ml.heads import GPT2HeadWithValueModel

%load_ext autoreload
%autoreload 2
%matplotlib inline

local_hugface_directory = '/Users/carson/projects/modelstates/hugface_models/'
#local_hugface_directory = '/home/carson/modelstates/hugface_models/'
os.listdir(local_hugface_directory )

torch.__version__ 1.12.1
NUM_GPUS 0
transformers.__version__ 4.21.1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['gpt2']

Some model sizes

'gpt2' num_params 124_439_808

'gpt2' with value head num_params 124_440_577

'gpt2-medium' num_params 354_823_168


https://github.com/huggingface/transformers/blob/v4.21.1/src/transformers/configuration_utils.py#L54

When you call `GPT2LMHeadModel.from_pretrained()`

```
class GPT2HeadWithValueModel(GPT2PreTrainedModel):

    """The GPT2HeadWithValueModel class implements a GPT2 language model with a secondary, scalar head."""

    def __init__(self, config):
        super().__init__(config)

        config.num_labels = 1
        self.transformer = GPT2Model(config)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.v_head = ValueHead(config)
        self.init_weights()

    def get_output_embeddings(self):
        return self.lm_head
```

the <class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'> config dict looks something like this:

```
GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.15.0",
  "use_cache": true,
  "vocab_size": 50257
}
```

https://huggingface.co/transformers/v4.11.3/_modules/transformers/models/gpt2/configuration_gpt2.html

summary is used for sequence compression

In [43]:
model_name = 'gpt2' # 'EleutherAI/gpt-j-6B' # 'coherence/gpt2-xl'
param_path = local_hugface_directory+model_name
print(param_path)

tokenizer = GPT2Tokenizer.from_pretrained(model_name,pad_token='<|endoftext|>')
#tokenizer = AutoTokenizer.from_pretrained(model_save_path,pad_token='<|endoftext|>')

/Users/carson/projects/modelstates/hugface_models/gpt2


```
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      .
      12 repeating blocks
      .
      (11): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
```

In [111]:
model = GPT2HeadWithValueModel.from_pretrained(
    model_name,
    cache_dir = param_path,
)

'''
model = GPT2LMHeadModel.from_pretrained(
    model_name,
    cache_dir = param_path,
)

model = GPTJForCausalLM.from_pretrained(
    model_save_path,
    revision="float16", 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True,
)
'''

print("num_params", sum(p.numel() for p in model.parameters() if p.requires_grad)) 

if NUM_GPUS == 1:
    print('model = model.cuda()')
    model = model.cuda()
elif NUM_GPUS > 1:
    # break up model and place model components on different GPUs
    print('model.parallelize()')
    model.parallelize()
else:
    print('did not place model on any GPUs')
    

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.4.attn.masked_bias', 'v_head.summary.bias', 'lm_head.weight', 'h.0.attn.masked_bias', 'h.3.attn.masked_bias', 'h.1.attn.masked_bias', 'v_head.summary.weight', 'h.9.attn.masked_bias', 'h.5.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.10.attn.masked_bias', 'h.6.attn.masked_bias', 'h.11.attn.masked_bias', 'h.2.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


num_params 124440577
did not place model on any GPUs


expected output from adding additional value head



class GPT2HeadWithValueModel(GPT2PreTrainedModel):

warning

```
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.6.attn.masked_bias', 'h.10.attn.masked_bias', 'h.0.attn.masked_bias', 'h.3.attn.masked_bias', 'h.7.attn.masked_bias', 'h.5.attn.masked_bias', 'h.11.attn.masked_bias', 'h.9.attn.masked_bias', 'h.8.attn.masked_bias', 'lm_head.weight', 'h.4.attn.masked_bias', 'v_head.summary.weight', 'h.2.attn.masked_bias', 'v_head.summary.bias', 'h.1.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
```
model
```
GPT2HeadWithValueModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      .
      12 repeating blocks
      .
      (11): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (v_head): ValueHead(
    (summary): Linear(in_features=768, out_features=1, bias=True)
    (activation): Identity()
    (first_dropout): Dropout(p=0.1, inplace=False)
    (last_dropout): Identity()
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
)
```

class GPT2HeadWithValueModel(GPT2LMHeadModel):

warning

```
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['v_head.summary.bias', 'v_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
```

model

```
GPT2HeadWithValueModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      .
      12 repeating blocks
      .
      (11): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (v_head): ValueHead(
    (summary): Linear(in_features=768, out_features=1, bias=True)
    (activation): Identity()
    (first_dropout): Dropout(p=0.1, inplace=False)
    (last_dropout): Identity()
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
)
```


In [112]:
input_txt = "I liked the movie Transformers!" + tokenizer.eos_token
input_ids = tokenizer.encode(input_txt, add_special_tokens=True, return_tensors="pt")
logits, transformer_outputs, values = model(input_ids)

In [113]:
logits = outputs.logits

In [114]:
pred_ids = torch.argmax(logits, dim=-1)

for i in range(input_ids.shape[1]):
    current_id = tokenizer.decode(input_ids[:, i])
    next_id = tokenizer.decode(pred_ids[:, i])
    print(current_id, '-->', next_id)

I --> .
 liked -->  the
 the -->  idea
 movie --> ,
 Transformers --> ,
! -->  I
<|endoftext|> --> The


In [48]:
def respond_to_batch(model, queries, txt_len=20, top_k=0, top_p=1.0):
    """Sample text from language model."""
    input_ids = queries
    for i in range(txt_len):
        # Get Logits
        outputs = model(input_ids)
        next_token_logits = outputs[0][:, -1, :]
        next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
        # Sample
        probs = F.softmax(next_token_logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1).squeeze(1)
        input_ids = torch.cat([input_ids, next_token.unsqueeze(-1)], dim=-1)
    return input_ids[:, -txt_len:]

In [54]:
query_txt_1 = "My most favourite movie is"
query_txt_2 = "My least favourite movie is"
queries_txt = [query_txt_1, query_txt_2]

queries = [tokenizer.encode(query_txt, return_tensors="pt") for query_txt in queries_txt]
print([q.shape for q in queries])
queries = torch.cat(queries)

responses = respond_to_batch(model, queries, txt_len=10)

for i in range(responses.shape[0]):
    response_txt = tokenizer.decode(responses[i])
    query_txt = queries_txt[i]
    print(query_txt + response_txt)

[torch.Size([1, 5]), torch.Size([1, 5])]
My most favourite movie is Rodney Dangerfield, whose character is tooker Danny
My least favourite movie is Spitfire. Luigi is an all-American comedic
